<a href="https://colab.research.google.com/github/DCajiao/Time-series-forecast-of-energy-consumption-in-Tetouan-City/blob/main/notebooks/03_MLP_modeling.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

### Entrenamiento, Predicción y Evaluación de un modelo MLP

#### **Importación de Datos**

In [1]:
import requests
import pandas as pd
import numpy as np

import seaborn as sns
import matplotlib.pyplot as plt
import matplotlib.dates as mdates

from io import BytesIO
from statsmodels.tsa.seasonal import seasonal_decompose
from statsmodels.tsa.stattools import adfuller, kpss

In [2]:
# Importar datos
DATA_GITHUB_URL = 'https://raw.githubusercontent.com/DCajiao/Time-series-forecast-of-energy-consumption-in-Tetouan-City/refs/heads/main/data/zone1_power_consumption_of_tetouan_city.csv'

# Descargar los datos desde github
response = requests.get(DATA_GITHUB_URL)

# Convertir en un df desde el xlsx de github
df = pd.read_csv(BytesIO(response.content), sep=',')

In [3]:
df

,datetime,temperature,humidity,wind_speed,general_diffuse_flows,zone_1
0,2017-01-01 00:00:00,6.559,73.8,0.083,0.051,34055.69620
1,2017-01-01 00:10:00,6.414,74.5,0.083,0.070,29814.68354
2,2017-01-01 00:20:00,6.313,74.5,0.080,0.062,29128.10127
3,2017-01-01 00:30:00,6.121,75.0,0.083,0.091,28228.86076
4,2017-01-01 00:40:00,5.921,75.7,0.081,0.048,27335.69620
...,...,...,...,...,...,...
52411,2017-12-30 23:10:00,7.010,72.4,0.080,0.040,31160.45627
52412,2017-12-30 23:20:00,6.947,72.6,0.082,0.051,30430.41825
52413,2017-12-30 23:30:00,6.900,72.8,0.086,0.084,29590.87452
52414,2017-12-30 23:40:00,6.758,73.0,0.080,0.066,28958.17490


#### **Entrenamiento**

---

#### **Predicción**

A un paso

A múltiples pasos

---

#### **Analisis de resultados**

Métricas a evaluar:
1. sMAPE
2. WAPE

---